# Expert System (i.e. Knowledge-Based System) Shell/Engine - Educational DEMO
Author: Ryan J Urbanowicz, PhD

Institution: Cedars Sinai Health Sciences University

Date: 1/16/2026

This notebook includes code assembling a **basic but flexible expert system shell, built from the ground up** (i.e. without using existing expert system shells like CLIPS, clipspy, or PyKE), but certainly directly inspired by them. 

This expert system shell is primarily intended for educational purposes. While it is a realtively simple implementation it has been designed with a good deal of flexibility - not found in other Python or Java expert system implementations which are often confusing, not well documented, and or no longer supported/used often. This shell can be used to create a fairly wide varity of expert and knowledge-based system decision making/reasoning tools. Here is a summary of the key features of this shell impelemntation. 
1. Handles both **deductive** (certain) and **inductive** (uncertain/probabilistic) **reasoning**
2. Can perform both **forward** and **backward chaining** (i.e. reasoning)
3. Separates the knowledge base (saved as a json file) from the inference engine and other components (a simple knowlege base editor and a simple explanation system)
4. Uses an easy-to-understand and simple syntax for facts, rules, and questions in the knoweldge-base (however potentially limiting for some applications)
5. Flexibly handles truth comparisons including **(==, >, <, <=, >=,!=)** and fact-states such as True/False, yes/no, etc. 
6. For inductive reasoning - employs certainty factores (0-1 values) - rule firing propagates uncertainty with a product of cfs, while 'and's' of conjuntive rules take the minimum cfs across rule conditions. When multiple rules fire, certainty factors are combined with the Mycin cf update (i.e. new_cf = new_cf + old_cf * (1 - new_cf))
7. The code in the shell below has also been adapted into a 'Streamlit' web dashboard/GUI that can be easily shared and played with. (i.e. **'expert_system_app.py'**)
    * This dashboard is:
        * Run with the command: **'streamlit run expert_system_app.py'**
        * Limited to **backward chaining**.
        * Allows loading of different knowlege bases as .json files.
        * Requires selection of **deductive** vs. **probailistic** (i.e. inductive) reasoning before loading a knowelege base.
        * Supports selection of certainty factors for inputs using a slider.
        * Allows selection from available goals in knowledge base that can be proved.
        * Provides a breakdown of reasoning, and an explanation of decisions.

## Expert System Shell
This shell does not be edited in order to build an expert system, however since this shell is simply and concisely built from the ground up, there are many opportunities to improve and extend this implementation to the needs of the user. 

In [1]:
import operator
import json

class KnowledgeManager:
    @staticmethod
    def load_from_json(engine, file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)

        # 1. Load Questions
        for fact_name, text in data.get("questions", {}).items():
            engine.add_question(fact_name, text)

        # 2. Load Facts
        for f_data in data.get("facts", []):
            try:
                engine.initialize_fact(f_data["name"],Fact(
                    f_data["name"],
                    f_data["value"],
                    f_data["cf"],
                    f_data["explanation"]
                ))
            except:
                engine.initialize_fact(f_data["name"],Fact(
                    f_data["name"],
                    f_data["value"],
                    1.0,
                    f_data["explanation"]
                ))

        # 3. Load Rules
        for r_data in data.get("rules", []):
            conditions = []
            for c_data in r_data["conditions"]:
                conditions.append(Condition(
                    c_data["fact1"], 
                    c_data["op"], 
                    c_data["fact2"],
                ))
            try:
                engine.add_rule(Rule(
                    r_data["id"],
                    conditions,
                    tuple(r_data["conclusion"]),
                    r_data["cf"],
                    r_data["explanation"]
                ))
            except:
                engine.add_rule(Rule(
                    r_data["id"],
                    conditions,
                    tuple(r_data["conclusion"]),
                    1.0,
                    r_data["explanation"]
                ))
        print(f"Successfully loaded {len(data['facts'])} facts, {len(data['rules'])} rules and {len(data['questions'])} questions.")

    @staticmethod
    def save_to_json(engine, file_path):
        # Optional: Logic to export current rules back to JSON
        data = {
            "questions": engine.questions,
            "facts": [
                {
                    "name": f.id,
                    "value": f.value,
                    "cf": f.cf,
                    "explanation": f.explanation
                } for f in engine.facts
            ],
            "rules": [
                {
                    "id": r.id,
                    "conditions": [vars(c) for c in r.conditions],
                    "conclusion": r.conclusion,
                    "cf": r.cf,
                    "explanation": r.explanation
                } for r in engine.rules
            ]
        }
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=2)


# Define supported operators for numerical comparison
OPERATORS = {
    ">": operator.gt,
    "<": operator.lt,
    ">=": operator.ge,
    "<=": operator.le,
    "==": operator.eq,
    "!=": operator.ne
}

class Condition:
    def __init__(self, fact1, op, fact2):
        self.fact1 = fact1 #can be a variable (starting with '$') or fixed value (not starting with '$)
        self.op = op # String: ">", "==", etc.
        self.fact2 = fact2 #can be a variable (starting with '$') or fixed value (not starting with '$)

    #cond.evaluate(fact1.value,fact2.value)
    def evaluate(self,fact1_value,fact2_value):
        # Perform the numerical or string comparison
        result = OPERATORS[self.op](fact1_value, fact2_value)
        return result


class Fact:
    def __init__(self, name, value, cf=1.0, explanation="Given as initial fact"):
        self.name = name
        self.value = value
        self.cf = cf
        self.explanation = explanation

    def __repr__(self):
        return f"Fact({self.name}={self.value}, CF={self.cf:.2f})"


class Rule:
    def __init__(self, id, conditions, conclusion, rule_cf, explanation):
        self.id = id
        self.conditions = conditions # conditions: list of (fact_name, value, is_negated), e.g., ("fever", "yes", True) means 'fever is NOT yes'
        self.conclusion = conclusion # (fact_name, value)
        self.rule_cf = rule_cf
        self.explanation = explanation


class ExpertSystem:
    def __init__(self, reasoning='deductive'):
        self.rules = []
        self.facts = {} # name -> Fact object
        self.questions = {} # name -> question text
        self.reasoning = reasoning

    def clear_facts(self):
        self.facts = {}
        
    def add_rule(self, rule):
        self.rules.append(rule)

    def initialize_fact(self, name, fact):
        self.facts[name] = fact

    def add_fact(self, name, value, cf=1.0, explanation="Initial"):
        new_fact = Fact(name, value, cf, explanation)
        if name in self.facts: #To update 
            # Combine certainty using MYCIN formula - If new evidence supports an existing hypothesis, the new CF is combined using (old_cf + cf * (1 - old_cf))
            old_cf = self.facts[name].cf
            self.facts[name].cf = old_cf + cf * (1 - old_cf)
        else:
            self.facts[name] = new_fact

    def add_question(self, fact_name, text):
        self.questions[fact_name] = text

    # --- (2) CERTAINTY & (5) EXPLANATION SYSTEM ---
    def get_explanation(self, fact_name):
        if fact_name not in self.facts:
            return "Fact not found."
        f = self.facts[fact_name]
        return f"Conclusion: {f.name} is {f.value} (Confidence: {f.cf:.2%})\nReasoning: {f.explanation}"

    # --- (3) FORWARD CHAINING ---
    def forward_chain(self):
        print("\n--- Forward Chaining (Numerical) ---")
        changed = True
        while changed:
            #print('NEW WHILE LOOP')
            changed = False
            for rule in self.rules:
                #print(f"Examining Rule: {rule.id} -> {rule.conclusion}")
                satisfied = True
                min_cf = 1.0
                for cond in rule.conditions:
                    #try to get both fact objects - if either facts are hard coded, don't bother checking for it's value
                    if cond.fact1.startswith('$') and cond.fact2.startswith('$'):
                        fact1 = self.facts.get(cond.fact1.removeprefix('$'))
                        fact2 = self.facts.get(cond.fact2.removeprefix('$'))
                    elif cond.fact1.startswith('$') and not cond.fact2.startswith('$'):
                        fact1 = self.facts.get(cond.fact1.removeprefix('$'))
                        self.add_fact(cond.fact2, cond.fact2, 1.0, 'Built-in fact')
                        fact2 = self.facts.get(cond.fact2)
                    elif not cond.fact1.startswith('$') and cond.fact2.startswith('$'):
                        self.add_fact(cond.fact1, cond.fact1, 1.0, 'Built-in fact')
                        fact1 = self.facts.get(cond.fact1)
                        fact2 = self.facts.get(cond.fact2.removeprefix('$'))
                    else:
                        print("ERROR:Rules cannot be defined based on two hard-coded facts, at least one must be a variable ('$') fact")

                    if fact1 and fact2 and cond.evaluate(fact1.value,fact2.value):
                        #min_cf = min(min_cf, float(fact1.cf * fact2.cf))
                        # Determining the certainty of rule condition based on the minimum certainty of all condition/facts in the rule (i.e. conjunctive rules)
                        min_cf = min(min_cf, fact1.cf, fact2.cf)
                    else:
                        satisfied = False; break

                if satisfied:
                    name, val = rule.conclusion
                    # Uncertainty propagated through rule firing (product of rule fact certainty and overall rule certainty)
                    new_cf = rule.rule_cf * min_cf
                    if name not in self.facts or self.facts[name].cf < new_cf:
                        self.add_fact(name, val, new_cf, rule.explanation)
                        print(f"Rule Fired: {rule.id} -> {name} is {val} with cf {new_cf:.2%}")
                        changed = True

    # --- (3) & (4) BACKWARD CHAINING WITH QUESTION LOOP ---
    def backward_chain(self, goal_name):
        print(f"\n--- Backward Chaining for Goal: {goal_name} ---")
        return self._prove(goal_name)
    
    def _prove(self, goal_name):
        if goal_name in self.facts: return self.facts[goal_name].cf

        rule_cfs = []
        for rule in self.rules:
            if rule.conclusion[0] == goal_name:
                rule_satisfied = True
                cond_cfs = [] # colection of fact certainties to prove current goal
                for cond in rule.conditions:
                    #for each fact in the rule either prove it recursively or note that its' a fact and needs no proof
                    res_cf = [1.0] #initialized with maximum certainty
                    #try to get both fact objects - if either facts are hard coded, don't bother checking for it's value
                    if cond.fact1.startswith('$') and cond.fact2.startswith('$'):
                        res_cf.append(self._prove(cond.fact1.removeprefix('$'))) #get combined confidence factor for the proof of this fact
                        fact1 = self.facts.get(cond.fact1.removeprefix('$'))
                        res_cf.append(self._prove(cond.fact2.removeprefix('$'))) #get combined confidence factor for the proof of this fact
                        fact2 = self.facts.get(cond.fact2.removeprefix('$')) 
                    elif cond.fact1.startswith('$') and not cond.fact2.startswith('$'):
                        res_cf.append(self._prove(cond.fact1.removeprefix('$'))) #get combined confidence factor for the proof of this fact
                        fact1 = self.facts.get(cond.fact1.removeprefix('$'))
                        res_cf.append(1.0)
                        self.add_fact(cond.fact2, cond.fact2, 1.0, 'Built-in fact')
                        fact2 = self.facts.get(cond.fact2)
                    elif not cond.fact1.startswith('$') and cond.fact2.startswith('$'):
                        res_cf.append(1.0)
                        self.add_fact(cond.fact1, cond.fact1, 1.0, 'Built-in fact')
                        fact1 = self.facts.get(cond.fact1)
                        res_cf.append(self._prove(cond.fact2.removeprefix('$'))) #get combined confidence factor for the proof of this fact
                        fact2 = self.facts.get(cond.fact2.removeprefix('$')) 
                    else:
                        print("ERROR:Rules cannot be defined based on two hard-coded facts, at least one must be a variable ('$') fact")

                    if min(res_cf) > 0 and fact1 and fact2 and cond.evaluate(fact1.value,fact2.value):
                        cond_cfs.append(min(res_cf)) # For rules nearest to ground facts - Determining the certainty of rule condition based on the minimum certainty of all condition/facts in the rule (i.e. conjunctive rules)
                    else:
                        rule_satisfied = False; break                  
                
                if rule_satisfied:
                    min_cond_cfs = min(cond_cfs) #For combining facts obtained from other proofss - Determining the certainty of rule condition based on the minimum certainty of all condition/facts in the rule (i.e. conjunctive rules)
                    new_cf = rule.rule_cf * min_cond_cfs # Uncertainty propagated through rule firing (product of rule fact certainty and overall rule certainty)
                    rule_cfs.append(new_cf)
                    self.add_fact(rule.conclusion[0], rule.conclusion[1], new_cf, rule.explanation)

        if rule_cfs:
            res = rule_cfs[0]
            for c in rule_cfs[1:]: res = res + c * (1 - res)
            return res

        if goal_name in self.questions:
            print(f"QUERY: {self.questions[goal_name]}")
            val_input = input("Value (e.g., 38.5 or 'yes'): ")
            # Try to convert to float if it's a number
            try: val = float(val_input)
            except: val = val_input.lower().strip()
            
            if self.reasoning == 'deductive':
                conf = 1.0
            else:
                conf = float(input("Confidence (0.0 to 1.0): "))
            self.add_fact(goal_name, val, conf, "User provided")
            return conf
        return 0.0



## Introduction to Knowledge Base Syntax
Below we explore simple expert systems designed as accessible examples.
* Each knowlege base is stored within a .json file. 
* The .json knowlege base syntax is that of a dictionary organized by 'questions', 'facts' and 'rules'. Each type of entry has its own required syntax.
    * 'questions' - a dictionary of key:value pairs where the key is the name of a fact needed by the system and the value is the text description of the input needed seen by the user.
    * 'facts' - an array of objects (i.e. a list of dictionaries) where each fact has the keys (name, value, cf, explanation) identifying the fact name, it's assigned value, it's certainty factor, and a description of the fact (used by the explanation system)
    * 'rules' - an array of objects (i.e. a list of dictionaries) where each rule has the keys (id, conditions, conclusion, cf, explanation) 
        * 'Conditions' are list objects that can include one or more conditions (to be satisfied) in order for the rule to 'fire' (i.e. add the conclusion as a new fact in the knowledge base)
        * Each condition has two 'facts' that are compared using any of the following (==, >, <, <=, >=,!=). 
        * These 'facts' can either be variables (starting with '$') (which can also be defined as facts in the KB directly) or static (hard-coded) values within the condition itself, that the system will automatically turn into 'implied facts'. 
        * 'Conclusions' are given as a tuple that first defines the [fact name, fact value] when the rule fires.
        * 'cf' and 'explanation' mean the same as for 'facts'

### Weather Knowlege Base (Example) - Deductive Reasoning - Forward Chaining
This is a simple expert system designed to provide users guidance on 'what to bring' when leaving the house based on current weather conditions.
* This knowledge base can be found in 'kb_weather.json'.
* In this forward chaining example, only facts and rules are used (not questions).
* Neither facts, rules, nor questions include certainty factors in this knowledge base, so the system will assume they all have a certainty factor of 1.0 by default.

The code below applies 'forward chaining' reasoning to this simple deductive reasoning example knowledge base.


In [2]:
# 1. Initialize the empty engine
engine = ExpertSystem('deductive') #deductive or probabilistic

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_weather.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Manually add some facts for testing forward chaining (i.e. part of the simple knowlege base editor)
engine.add_fact("precipitation", "yes")
engine.add_fact("windy", "yes") 
engine.add_fact("temperature", 80)

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "what to bring"

# 3. Use the system for a decision making
print("\n--- AUTOMATED DECISION MAKING ---")

# Forward chaining will now use the rules and facts loaded from JSON
engine.forward_chain()

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

#print(engine.facts) #optionally print all facts stored in the knowlege base at the end of forward chaining
engine.clear_facts() #cleanup expert system fact memory for another run

Successfully loaded 3 facts, 9 rules and 3 questions.

--- AUTOMATED DECISION MAKING ---

--- Forward Chaining (Numerical) ---
Rule Fired: R_RAIN_PROTECT -> rain_protect is True with cf 100.00%
Rule Fired: R_RAINCOAT -> what to bring is raincoat with cf 100.00%

Conclusion: what to bring is raincoat (Confidence: 100.00%)
Reasoning: Bring raincoat when it's raining and windy.


### Weather Knowlege Base (Example) - Deductive Reasoning - Backward Chaining
The code below applies 'backward chaining' reasoning to this simple deductive reasoning example knowledge base. When run, a prompt will appear asking for user inputs generated by the questions built into the knowlege base. This example will use everything in the knowledge base, i.e. facts, rules, and questions.

In [3]:
# 1. Initialize the empty engine
engine = ExpertSystem('deductive')

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_weather.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "what to bring"

# 3. Use the system for a diagnosis
print("\n--- MODULAR DIAGNOSTIC SESSION ---")
# Backward chaining will now use the rules and questions loaded from JSON
engine.backward_chain(goal_name)

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

Successfully loaded 3 facts, 9 rules and 3 questions.

--- MODULAR DIAGNOSTIC SESSION ---

--- Backward Chaining for Goal: what to bring ---
QUERY: Is it currently precipitating? (yes/no)
QUERY: What is the current temperature (in Fahrenheit)
QUERY: Is it currently windy? (yes/no)

Conclusion: what to bring is umbrella (Confidence: 100.00%)
Reasoning: Bring an umbrella when it's raining but not windy.


## Simple Diagnosis Knowlege Base (Example) - Inductive Reasoning - Forward Chaining
This is a simple expert system designed to provide users guidance on 'diagnosis' of flu.
* This knowledge base can be found in 'kb_diagnose_flu.json'.
* In this forward chaining example, only facts and rules are used (not questions).
* Facts, and rules include certainty factors in this knowledge base denoted by 'cf'.
* Users will input certainty factors along with any input facts added at the start of forward chaining

The code below applies 'forward chaining' reasoning to this simple inductive reasoning example knowledge base.

In [4]:
# 1. Initialize the empty engine
engine = ExpertSystem('probabilistic') #deductive or probabilistic

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_diagnose_flu.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Manually add some facts for testing forward chaining (i.e. part of the simple knowlege base editor)
engine.add_fact("temperature", 101, 1.0)
engine.add_fact("cough", "yes", 1.0) 

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "diagnosis"

# 3. Use the system for a decision making
print("\n--- AUTOMATED DECISION MAKING ---")

# Forward chaining will now use the rules and facts loaded from JSON
engine.forward_chain()

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

#print(engine.facts) #optionally print all facts stored in the knowlege base at the end of forward chaining
engine.clear_facts() #cleanup expert system fact memory for another run

Successfully loaded 1 facts, 2 rules and 3 questions.

--- AUTOMATED DECISION MAKING ---

--- Forward Chaining (Numerical) ---
Rule Fired: R_FEVER -> status is feverish with cf 90.00%
Rule Fired: R_FLU -> diagnosis is influenza with cf 72.00%

Conclusion: diagnosis is influenza (Confidence: 72.00%)
Reasoning: Fever combined with a cough suggests Influenza.


## Simple Diagnosis Knowlege Base (Example) - Inductive Reasoning - Backward Chaining
The code below applies 'backward chaining' reasoning to this simple deductive reasoning example knowledge base. When run, a prompt will appear asking for user inputs generated by the questions built into the knowlege base. This example will use everything in the knowledge base, i.e. facts, rules, and questions.
* Users will input certainty factors (cf) along with any input questions asked at the start of backward chaining

The code below applies 'backward chaining' reasoning to this simple inductive reasoning example knowledge base.

In [8]:
# 1. Initialize the empty engine
engine = ExpertSystem('probabilistic')

# 2. Load the Knowledge (Assuming you saved the JSON above as 'kb.json')
# For this demo, we'll simulate the file loading
try:
    KnowledgeManager.load_from_json(engine, 'kb_diagnose_flu.json')
except FileNotFoundError:
    print("Error: Knowledge base file not found.")

# Specify the 'goal' (i.e. target fact) we want to determine from the knowlege base
goal_name = "diagnosis"

# 3. Use the system for a diagnosis
print("\n--- MODULAR DIAGNOSTIC SESSION ---")
# Backward chaining will now use the rules and questions loaded from JSON
engine.backward_chain(goal_name)

# 4. Display Explained Results
print("\n" + "="*40)
print(engine.get_explanation(goal_name))
print("="*30)

Successfully loaded 1 facts, 2 rules and 3 questions.

--- MODULAR DIAGNOSTIC SESSION ---

--- Backward Chaining for Goal: diagnosis ---
QUERY: What is the patient's temperature in Farenheit?
QUERY: Does the patient have a persistent cough? (yes/no)

Conclusion: diagnosis is influenza (Confidence: 64.00%)
Reasoning: Fever combined with a cough suggests Influenza.
